In [75]:
import pandas as pd
import plotly.express as px
import nbformat
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score


import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score



In [76]:
DinningPred = pd.read_excel("DinningData.xlsx", sheet_name='Data for 2025 pred')


In [77]:
DinningPred.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          285 non-null    int64  
 1   Semester     285 non-null    int64  
 2   Month        285 non-null    int64  
 3   Year         285 non-null    int64  
 4   Hours        240 non-null    float64
 5   Money Spent  285 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 13.5 KB


In [78]:
missing_values = DinningPred.isnull().sum()
missing_values


DEP             0
Semester        0
Month           0
Year            0
Hours          45
Money Spent     0
dtype: int64

In [79]:
print(DinningPred.columns)

Index(['DEP', 'Semester', 'Month', 'Year', 'Hours', 'Money Spent'], dtype='object')


In [80]:
# def split_dataset(df):
#     """
#     Splits the dataset into training and testing sets.
    
#     Parameters:
#     - df (pd.DataFrame): The input dataset.
    
#     Returns:
#     - X_train, X_test, y_train, y_test: Training and testing sets for features and labels.
#     """
#     # Replace 'your_target_column' with the actual name of your target column
#     target_column = 'Hours'
    
#     X = df.drop(target_column, axis=1)
#     y = df[target_column]
    
#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
#     print(f'Size of the Train: {X_train.shape[0]}')
#     print(f'Size of the Test: {X_test.shape[0]}')
    
#     return X_train, X_test, y_train, y_test


# def train_model_random_forest(X_train, X_test, y_train, y_test):
#     """
#     Creates and trains a RandomForest model on the given training set 
#     and evaluates its performance on the test set.
    
#     Parameters:
#     - X_train, X_test, y_train, y_test: Training and testing sets for features and labels.
    
#     Returns:
#     - accuracy (float): Accuracy score of the trained model on the test set.
#     """
#     # Create and train the RandomForest model
#     rf_model = RandomForestClassifier()
#     rf_model.fit(X_train, y_train)

#     # Evaluate the model on the test set
#     y_pred = rf_model.predict(X_test)
#     accuracy = accuracy_score(y_test, y_pred)
#     print(f"Accuracy: {accuracy}\n")
#     return accuracy

# accuracies = {}


In [81]:
def linear_regression_impute(df, target_column, feature_columns, tolerance=0.1):
    # Separate the data into rows with and without missing values
    df_missing = df[df[target_column].isnull()]
    df_not_missing = df.dropna(subset=[target_column])

    # If there are no missing values, return the original dataframe
    if df_missing.empty:
        print(f"No missing values for {target_column}. Skipping imputation.")
        return df, None, None, None, None  # Return None for metrics

    # Split df_not_missing into training and testing sets
    X = df_not_missing[feature_columns]
    y = df_not_missing[target_column]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Create and fit a linear regression model
    model = LinearRegression()
    model.fit(X_train, y_train)

    # Predict on the test set to evaluate the model
    y_pred = model.predict(X_test)

    # Calculate evaluation metrics
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)


    print(f"Evaluation metrics for {target_column}:")
    print(f"Mean Squared Error (MSE): {mse:.2f}")
    print(f"Mean Absolute Error (MAE): {mae:.2f}")
    print(f"R-squared (R²): {r2:.2f}")

    # Predict missing values using the model
    X_missing = df_missing[feature_columns]
    predicted_values = model.predict(X_missing)

    # Create a dataset with imputed missing values
    df_filled = df.copy()
    df_filled.loc[df_missing.index, target_column] = predicted_values

    return df_filled, mse, mae, r2

In [82]:
target_columns_to_impute = ['Hours']
feature_columns_for_imputation = ['Year', 'Month', 'DEP', 'Semester']
linear_data = DinningPred.copy()

In [83]:
metrics = {}

In [85]:
for target_column in target_columns_to_impute:
    linear_data, mse, mae, r2 = linear_regression_impute(linear_data, target_column, feature_columns_for_imputation)
    if mse is not None and mae is not None and r2 is not None:
        metrics[target_column] = {
            'MSE': mse,
            'MAE': mae,
            'R-squared': r2
        }

Evaluation metrics for Hours:
Mean Squared Error (MSE): 113299.04
Mean Absolute Error (MAE): 227.68
R-squared (R²): 0.57


In [86]:
# Print all metrics
for target_column, metric in metrics.items():
    print(f"Metrics for {target_column}:")
    print(f"  Mean Squared Error (MSE): {metric['MSE']:.2f}")
    print(f"  Mean Absolute Error (MAE): {metric['MAE']:.2f}")
    print(f"  R-squared (R²): {metric['R-squared']:.2f}")

Metrics for Hours:
  Mean Squared Error (MSE): 113299.04
  Mean Absolute Error (MAE): 227.68
  R-squared (R²): 0.57


In [87]:
print(linear_data)


     DEP  Semester  Month  Year        Hours  Money Spent
0      1         1      9  2020  3859.980000   31844.8350
1      1         1     10  2020  4722.980000   38964.5850
2      1         1     11  2020  2842.680000   23452.1100
3      1         1     12  2020  1348.500000   11125.1250
4      1         2      1  2021  2247.990000   18545.9175
..   ...       ...    ...   ...          ...          ...
280    5         2      1  2025   -53.657769       0.0000
281    5         3      2  2025  -327.595303       0.0000
282    5         3      3  2025  -363.841731       0.0000
283    5         3      4  2025  -400.088158       0.0000
284    5         3      5  2025  -436.334586       0.0000

[285 rows x 6 columns]


In [88]:
missing_values = linear_data.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64

In [89]:
df_2025 = linear_data[linear_data['Year'] == 2025]
df_2025


,DEP,Semester,Month,Year,Hours,Money Spent
52,1,2,1,2025,1089.508626,0.0
53,1,3,2,2025,815.571092,0.0
54,1,3,3,2025,779.324665,0.0
55,1,3,4,2025,743.078237,0.0
56,1,3,5,2025,706.831809,0.0
109,2,2,1,2025,803.717027,0.0
110,2,3,2,2025,529.779493,0.0
111,2,3,3,2025,493.533066,0.0
112,2,3,4,2025,457.286638,0.0
113,2,3,5,2025,421.040211,0.0


In [90]:
linear_data = linear_data.applymap(lambda x: max(0, x))


/var/folders/b5/c2s8x6hx5bz57tz7p2pf4f2h0000gn/T/ipykernel_74203/2910141043.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  linear_data = linear_data.applymap(lambda x: max(0, x))


In [91]:
df_2024 = linear_data[linear_data['Year'] == 2024]
df_2024

,DEP,Semester,Month,Year,Hours,Money Spent
40,1,2,1,2024,948.630000,8774.8275
41,1,3,2,2024,1057.590000,9782.7075
42,1,3,3,2024,694.670000,6425.6975
43,1,3,4,2024,1065.520000,9856.0600
44,1,3,5,2024,418.670000,3872.6975
45,1,4,6,2024,0.000000,0.0000
46,1,4,7,2024,0.000000,0.0000
47,1,4,8,2024,0.000000,0.0000
48,1,1,9,2024,1145.012715,0.0000
49,1,1,10,2024,1108.766287,0.0000


In [92]:
df_2025 = linear_data[linear_data['Year'] == 2025]
df_2025

,DEP,Semester,Month,Year,Hours,Money Spent
52,1,2,1,2025,1089.508626,0.0
53,1,3,2,2025,815.571092,0.0
54,1,3,3,2025,779.324665,0.0
55,1,3,4,2025,743.078237,0.0
56,1,3,5,2025,706.831809,0.0
109,2,2,1,2025,803.717027,0.0
110,2,3,2,2025,529.779493,0.0
111,2,3,3,2025,493.533066,0.0
112,2,3,4,2025,457.286638,0.0
113,2,3,5,2025,421.040211,0.0


In [93]:
PredictedData = linear_data.copy()


In [94]:
PredictedData

,DEP,Semester,Month,Year,Hours,Money Spent
0,1,1,9,2020,3859.98,31844.8350
1,1,1,10,2020,4722.98,38964.5850
2,1,1,11,2020,2842.68,23452.1100
3,1,1,12,2020,1348.50,11125.1250
4,1,2,1,2021,2247.99,18545.9175
...,...,...,...,...,...,...
280,5,2,1,2025,0.00,0.0000
281,5,3,2,2025,0.00,0.0000
282,5,3,3,2025,0.00,0.0000
283,5,3,4,2025,0.00,0.0000


In [95]:
PredictedData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 285 entries, 0 to 284
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   DEP          285 non-null    int64  
 1   Semester     285 non-null    int64  
 2   Month        285 non-null    int64  
 3   Year         285 non-null    int64  
 4   Hours        285 non-null    float64
 5   Money Spent  285 non-null    float64
dtypes: float64(2), int64(4)
memory usage: 13.5 KB


In [96]:
missing_values = PredictedData.isnull().sum()
missing_values

DEP            0
Semester       0
Month          0
Year           0
Hours          0
Money Spent    0
dtype: int64